## References

- https://docs.dask.org/

In [1]:
from IPython.display import Image

## Single Machine: dask.distributed

In [2]:
from dask.distributed import Client
client = Client()
client

/home/developer/miniconda3/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38995 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:44883 Dashboard: http://127.0.0.1:38995/status,Cluster Workers: 3 Cores: 6 Memory: 67.26 GB


## dask-scheduler

A dask.distributed network consists of one dask-scheduler process and several dask-worker processes that connect to that scheduler. These are normal Python processes that can be executed from the command line. We launch the dask-scheduler executable in one process and the dask-worker executable in several processes, possibly on different machines

https://docs.dask.org/en/latest/setup/cli.html

<img src="img/Screen Shot 2021-01-03 at 00.37.38.png">

## Why Dask?

- It provide method to scale up Pandas, Scikit-Learn, and Numpy with native feature and minimal rewriting
- Dask can enable efficient parallel computations on single machines by leveraging their multi-core CPUs and streaming data efficiently from disk [1]
- Smaller and lighter weight than Spark [2]


[1] https://docs.dask.org/en/latest/why.html

[2] https://docs.dask.org/en/latest/spark.html

## DataFrames: Read and Write Data

In [3]:
# from dask.distributed import Client
# client = Client(n_workers=1, threads_per_worker=6, processes=False, memory_limit='60GB')
# client

In [4]:
cell_ref_path = "data/cell.csv"
event_input_path = "data/data_1gb.csv"
output_path = "output/mapped_cells_1gb.csv"

### Create output in HDF5

In [5]:
%%time
import dask.dataframe as dd
cell_input = dd.read_csv(cell_ref_path)
event_input = dd.read_csv(event_input_path)

mapped_cells = dd.merge(event_input,cell_input,left_on='mno_cell_id',right_on='mno_cell_id').compute()

mapped_cells = mapped_cells[['mno_ms_id','pos_time','cell_id']]
mapped_cells = mapped_cells.rename(columns={'cell_id': 'mno_cell_id'})

mapped_cells.to_hdf(output_path + ".hdf5", '/data')

CPU times: user 15 s, sys: 3.18 s, total: 18.2 s
Wall time: 28.2 s


### Create output in CSV

In [6]:
%%time
import dask.dataframe as dd
cell_input = dd.read_csv(cell_ref_path)
event_input = dd.read_csv(event_input_path)

mapped_cells = dd.merge(event_input,cell_input,left_on='mno_cell_id',right_on='mno_cell_id').compute()

mapped_cells = mapped_cells[['mno_ms_id','pos_time','cell_id']]
mapped_cells = mapped_cells.rename(columns={'cell_id': 'mno_cell_id'})

mapped_cells.to_csv(output_path, index=False)

CPU times: user 48.1 s, sys: 3.71 s, total: 51.9 s
Wall time: 59.6 s


## Conclusion

- In Dask, writing an output file in HDF5 was much faster than CSV file 
- Without any much setup, Dask already doing the data processing in multi-processing